In [66]:
import pandas as pd 
import requests 
from plyer import notification
from datetime import datetime
import notificacao
import sqlite3
import re


# Extraindo a tabela banks da API
url = "https://brasilapi.com.br/api/banks/v1"

bancos = requests.get(url)

data = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

if bancos.status_code == 200:
    bancos_json = bancos.json()

# Transformando as tabelas em um DataFrame
bancos_full = pd.DataFrame(bancos_json)

# Excluindo linhas com a coluna code que possuem missing values
bancos_full = bancos_full.dropna(subset=['code'])

#Transformando os dados da coluna code de float para int
bancos_full = bancos_full.astype({'code':int})

# Conectando com banco de dados e salvando os dados brutos
bancos_connect = sqlite3.connect('Projeto Final.db')
bancos_full.to_sql('Bancos Bruto', bancos_connect, if_exists='replace', index=False )

# Alterando o nome das colunas
bancos_full_renomeado = bancos_full.set_axis(['ISPB', 'Nome', 'Código', 'Nome Completo'], axis=1)

# Alterando a coluna Nome Completo para deixar todos nomes em maiusculo
bancos_full_renomeado['Nome Completo']=bancos_full_renomeado['Nome Completo'].str.upper()

notificacao.alerta(bancos, data, url)

#Tratamento de Missing Values
bancos_full_tratado = (bancos_full_renomeado.dropna())

# Ordenação pela coluna Código
bancos_full_tratado.sort_values('Código')

# Salvando os dados tratados no banco de dados e fechando a conexão
bancos_full_tratado.to_sql('Bancos Tratado', bancos_connect, if_exists='replace', index=False )
bancos_connect.close()






In [74]:
# Extraindo a tabela corretoras da API
url = "https://brasilapi.com.br/api/cvm/corretoras/v1"

corretoras = requests.get(url)

data = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

if corretoras.status_code == 200:
    corretoras_json = corretoras.json()

# Transformando as tabelas em um DataFrame
corretoras_full = pd.DataFrame(corretoras_json)

# Conectando com banco de dados e salvando os dados brutos
corretoras_connect = sqlite3.connect('Projeto Final.db')
corretoras_full.to_sql('Corretoras Bruto', corretoras_connect, if_exists='replace', index=False )

# Alterando o nome das colunas
corretoras_full_renomeado = corretoras_full.set_axis(['CNPJ', 'Tipo', 'Razão Social', 'Nome Comercial', 'Status', 'E-Mail', 'Telefone',
                          'CEP', 'Pais', 'UF', 'Municipio', 'Bairro', 'Complemento', 'Logradouro', 'Data Patrimonio', 
                          'Valor Patrimonio', 'Codigo CVM', 'Data Inicio', 'Data Registro'], axis=1)


notificacao.alerta(corretoras, data, url)

# Formatacao da coluna valor patrimonio

def formatar(valor):
    return "R${:,.2f}".format(valor)

corretoras_full_renomeado['Valor Patrimonio'] = pd.to_numeric(corretoras_full_renomeado['Valor Patrimonio'])
corretoras_full_renomeado['Valor Patrimonio'] = corretoras_full_renomeado['Valor Patrimonio'].apply(formatar)

# Formatacao das colunas com datas

corretoras_full_renomeado['Data Registro'] = pd.to_datetime(corretoras_full_renomeado['Data Registro'])
corretoras_full_renomeado['Data Registro'] = corretoras_full_renomeado['Data Registro'].dt.strftime('%d-%m-%Y')


# Exclusão de colunas irrelevantes para a análise. 

corretoras_full_colunas = corretoras_full_renomeado.drop(['Tipo','E-Mail', 'CEP', 'Complemento', 'Logradouro','Data Inicio','Data Patrimonio'], axis=1)

# Ajustando o Formato do CNPJ
padrao = r'(\d{2})(\d{3})(\d{3})(\d{4})(\d{2})'
corretoras_full_colunas['CNPJ'] = corretoras_full_colunas['CNPJ'].str.replace(padrao, r'\1.\2.\3/\4-\5', regex = True)

# Ajustando Formato do Telefone
filtro_tel =  corretoras_full_colunas['Telefone'].str.len() < 8 
corretoras_full_colunas.loc[filtro_tel, 'Telefone'] = '3'+ corretoras_full_colunas.loc[filtro_tel, 'Telefone']
padrao_tel = r'(\d{4})(\d{4})'
corretoras_full_colunas['Telefone'] = corretoras_full_colunas['Telefone'].str.replace(padrao_tel, r'\1-\2', regex=True)

# Excluindo corretoras que não possuem Telefone na base de dados
filtro_sem_tel =corretoras_full_colunas[ corretoras_full_colunas['Telefone'].str.len() == 1].index 
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_sem_tel)

#Excluindo Corretoras que não possuem Pais na base de dados
filtro_pais = corretoras_full_colunas[corretoras_full_colunas['Pais'].str.len() == 0].index
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_pais)

#Excluindo Corretoras que não possuem Municipio na base de dados
filtro_municipio = corretoras_full_colunas[corretoras_full_colunas['Municipio'].str.len() == 0].index
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_municipio)

#Excluindo Corretoras que não possuem UF na base de dados
filtro_UF = corretoras_full_colunas[corretoras_full_colunas['UF'].str.len() == 0].index
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_UF)

#Excluindo Corretoras que não possuem Bairro na base de dados
filtro_bairro = corretoras_full_colunas[corretoras_full_colunas['Bairro'].str.len() == 0].index
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_bairro)

#Excluindo corretoras que não possuem nome comercial
filtro_nome_comercial = corretoras_full_colunas[corretoras_full_colunas['Nome Comercial'].str.len() == 0].index
filtro_nome = corretoras_full_colunas[corretoras_full_colunas['Nome Comercial'] == '--'].index
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_nome_comercial)
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_nome)

#Excluindo corretoras que possuem valor de patrimonio zerado
filtro_valor = corretoras_full_colunas[corretoras_full_colunas['Valor Patrimonio'] == 'R$0.00'].index
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_valor)

# Filtrando as corretoras que estão em funcionamento
corretoras_ativas = corretoras_full_colunas.loc[corretoras_full_colunas['Status'] =='EM FUNCIONAMENTO NORMAL']

# Filtrando as corretoras que estão canceladas
corretoras_canceladas = corretoras_full_colunas.loc[corretoras_full_colunas['Status'] =='CANCELADA']

# Salvando os dados tratados no banco de dados e fechando a conexão
corretoras_full_colunas.to_sql('Corretoras Tratado', corretoras_connect, if_exists='replace', index=False )
corretoras_ativas.to_sql('Corretoras Tratado Ativas', corretoras_connect, if_exists='replace', index=False )
corretoras_canceladas.to_sql('Corretoras Tratado Canceladas', corretoras_connect, if_exists='replace', index=False )
bancos_connect.close()

corretoras_full_colunas.info()


<class 'pandas.core.frame.DataFrame'>
Index: 147 entries, 0 to 372
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   CNPJ              147 non-null    object
 1   Razão Social      147 non-null    object
 2   Nome Comercial    147 non-null    object
 3   Status            147 non-null    object
 4   Telefone          147 non-null    object
 5   Pais              147 non-null    object
 6   UF                147 non-null    object
 7   Municipio         147 non-null    object
 8   Bairro            147 non-null    object
 9   Valor Patrimonio  147 non-null    object
 10  Codigo CVM        147 non-null    object
 11  Data Registro     147 non-null    object
dtypes: object(12)
memory usage: 14.9+ KB


In [50]:

# Extraindo a tabela PIX da API
url = 'https://brasilapi.com.br/api/pix/v1/participants'

pix = requests.get(url)

data = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

if pix.status_code == 200:
    pix_json = pix.json()

# Transformando as tabelas em um DataFrame
pix_full = pd.DataFrame(pix_json)

# Conectando com banco de dados e salvando os dados brutos
pix_connect = sqlite3.connect('Projeto Final.db')
pix_full.to_sql('Pix Bruto', pix_connect, if_exists='replace', index=False )

# Alterando o nome das colunas
pix_full_renomeado = pix_full.set_axis(['ISPB', 'Nome', 'Nome Reduzido', 'Modalidade', 'Tipo', 'Inicio'], axis=1)

notificacao.alerta(pix, data, url)

# Tratamento da coluna Inicio que contem data e hora
pix_full_renomeado['Inicio'] = pd.to_datetime(pix_full_renomeado['Inicio'])
pix_full_renomeado['Inicio'] = pix_full_renomeado['Inicio'].dt.strftime('%d-%m-%Y %H:%M')

# Salvando os dados tratados no banco de dados e fechando a conexão
pix_full_renomeado.to_sql('Pix Tratado', pix_connect, if_exists='replace', index=False )
pix_connect.close()


pix_full_renomeado


,ISPB,Nome,Nome Reduzido,Modalidade,Tipo,Inicio
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,03-11-2020 09:30
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,03-11-2020 09:30
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,03-11-2020 09:30
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,03-11-2020 09:30
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,03-11-2020 09:30
...,...,...,...,...,...,...
799,95213211,"COOPERATIVA DE CRéDITO, POUPANçA E INVESTIMENT...",COOP SICREDI CAMINHO DAS ÁGUAS RS,PDCT,IDRT,03-11-2020 09:30
800,95424891,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...",COOP SICREDI VALE DO RIO PARDO,PDCT,IDRT,03-11-2020 09:30
801,95594941,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...",COOP SICREDI REG CENTRO RS/MG,PDCT,IDRT,03-11-2020 09:30
802,97259253,COOPERATIVA E ECONOMIA E CRÉDITO MÚTUO DOS MÉD...,CECM MÉDICOS PLANALTO MÉDIO RS - CREDISIS/CRED...,PDCT,IDRT,04-10-2021 12:53
